In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd

In [20]:
import sys
sys.path.append('./models') 
from gaussian import GP
from gaussian import RBF
from bandit import Bandit

sys.path.append('./train_data') 
from train_func import TrueFunc

In [21]:
# func_sigma_list = [i/10 for i in range(1, 6,1)]
# func_max_list  = [i/10 for i in range(2, 12,2)]

# gp_noise_list   = [i/100 for i in range(5, 35, 5)]
# rbf_alpha_list  = [i/100 for i in range(5, 35, 5)]
# rbf_beta_list   = [i/100 for i in range(5, 35, 5)] 

In [28]:
GP_ME = 0.5
n_play = 100
n_sample = 100

func_sigma_list = [i/10 for i in range(1, 6,3)]
func_max_list  = [i/10 for i in range(2, 12,6)]

gp_noise_list   = [i/100 for i in range(5, 35, 20)]
rbf_alpha_list  = [i/100 for i in range(5, 35, 20)]
rbf_beta_list   = [i/100 for i in range(5, 35, 20)] 

In [23]:
def get_max_seed(dir_path):
    files = os.listdir(dir_path)
    seed_files = [f for f in files if 'seed_' in f]
    seed_numbers = [int(f.split('seed_')[1].split('.')[0]) for f in seed_files]
    if not seed_numbers:
        return 0
    return  max(seed_numbers)

In [24]:
def play_bandit_gp_ts():
    
    dir_path = '../results/csv/gp_ts/'
    seed = get_max_seed(dir_path) + 1
    
    data = []
    for train_func_sigma in func_sigma_list:
        for train_func_max in func_max_list:
            for gp_noise in gp_noise_list:
                for rbf_alpha in rbf_alpha_list:
                    for rbf_beta in rbf_beta_list:
                        
                        train_func = TrueFunc(train_func_sigma, train_func_max).peak_one
                        kernel = RBF(rbf_beta, rbf_alpha)
                        model = GP(GP_ME, gp_noise, kernel)
                        
                        np.random.seed(seed)
                        bandit = Bandit(n_play, n_sample)
                        train, reward = bandit.gp_ts(train_func, model)

                        data.append([train_func_sigma, train_func_max, gp_noise, rbf_alpha, rbf_beta, sum(reward)])                     
                        column=['f_sigma', 'f_max', 'noise', 'alpha','beta','reward']
                        df = pd.DataFrame(data, columns=column)
                        csv_name = "gp_ts_seed_"+ str(seed) + ".csv"
                        df.to_csv(str(dir_name) + str(csv_name))
                        
    print(f'finished seed is {seed}')

In [29]:
n = 1
for _ in range(n):
    play_bandit_gp_ts()   

finished seed is 1
